In [60]:
# %cd /content/drive/MyDrive/Fall23/Capstone Research/new_efforts

In [61]:
# !pip install torchmetrics

In [62]:
# !python3 "/content/drive/MyDrive/Fall23/Capstone Research/new_efforts/ITU-ML5G-PS-007-GNN-m0b1us/std_train.py" -ds CBR+MB --ckpt-path moblus_mb


In [63]:
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import warnings

from torchmetrics.regression import MeanAbsolutePercentageError
from tqdm import tqdm

import numpy as  np

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# from torchrec import JaggedTensor

In [64]:
ds_path = "extracted_moblus_devices_cv/3"
ds_val = "extracted_moblus_devices_cv/3"
ds_train = tf.data.Dataset.load(f"{ds_path}/training", compression="GZIP")
ds_val = tf.data.Dataset.load(f"{ds_path}/validation", compression="GZIP")


In [65]:

# Convert TF dataset to PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, tf_dataset):
        self.tf_dataset = tf_dataset

    def __len__(self):
        # Calculate the length of the dataset
        # Implement according to the structure of your dataset
        return len(self.tf_dataset)

    def __getitem__(self, idx):
        # Retrieve an item from the dataset
        # Implement according to the structure of your dataset
        return self.tf_dataset[idx]

In [66]:
train_list = []
# X_graphs = []
for item in ds_train:
    # Assuming the elements in the dataset are tensors, convert them into a format compatible with PyTorch
    train_list.append((item[0], item[1].numpy()))

val_list = []
# X_graphs = []
for item in ds_val:
    # Assuming the elements in the dataset are tensors, convert them into a format compatible with PyTorch
    val_list.append((item[0], item[1].numpy()))


In [67]:
mean_std_scores_fields = {
        "flow_traffic",
        "flow_packets",
        "flow_pkts_per_burst",
        "flow_bitrate_per_burst",
        "flow_packet_size",
        "flow_p90PktSize",
        "rate",
        "flow_ipg_mean",
        "ibg",
        "flow_ipg_var",
        "link_capacity",
    }

In [68]:
mean_std_scores_fields

{'flow_bitrate_per_burst',
 'flow_ipg_mean',
 'flow_ipg_var',
 'flow_p90PktSize',
 'flow_packet_size',
 'flow_packets',
 'flow_pkts_per_burst',
 'flow_traffic',
 'ibg',
 'link_capacity',
 'rate'}

In [69]:
def get_mean_std_dict(
    ds: tf.data.Dataset, params, include_y = None
):
    """Get the min and the max-min for different parameters of a dataset. Later used by the models for the min-max normalization.

    Parameters
    ----------
    ds : tf.data.Dataset
        Training dataset where to base the min-max normalization from.

    params : List[str]
        List of strings indicating the parameters to extract the features from.

    include_y : Optional[str], optional
        Indicates if to also extract the features of the output variable.
        Inputs indicate the string key used on the return dict. If None, it is not included.
        By default None.

    Returns
    -------
    Dict[str, Tuple[np.ndarray, np.ndarray]]
        Dictionary containing the values needed for the min-max normalization.
        The first value is the min value of the parameter, and the second is 1 / (max - min).
    """

    # Use first sample to get the shape of the tensors
    iter_ds = iter(ds)
    sample, label = next(iter_ds)
    params_lists = {param: sample[param].numpy() for param in params}
    if include_y:
        params_lists[include_y] = label.numpy()

    # Include the rest of the samples
    for sample, label in iter_ds:
        for param in params:
            params_lists[param] = np.concatenate(
                (params_lists[param], sample[param].numpy()), axis=0
            )
        if include_y:
            params_lists[include_y] = np.concatenate(
                (params_lists[include_y], label.numpy()), axis=0
            )

    scores = dict()
    for param, param_list in params_lists.items():
        mean_val = np.mean(param_list, axis=0)
        std_val = np.std(param_list, axis=0)

        if all(std_val) == 0:
            scores[param] = [mean_val, std_val]
        else:
            scores[param] = [mean_val, 1/std_val]

    return scores

In [70]:
ibgs = []
flow_p10PktSize  = []
flow_p50PktSize = []
flow_p90PktSize =  []
rate = []
flow_ipg_var = []
flow_packet_size = []
flow_packets = []
flow_type = []
flow_pkts_per_burst = []
flow_bitrate_per_burst = []

flow_ipg_mean = []
flow_traffic = []
link_capacity = []
flow_length = []
max_link_load = []
path_to_link = []
link_to_path = []

delays =  []

new_data =[]
z_start = 0
for z in range(16, len(train_list[:]),16):
  ibgs = []
  flow_p10PktSize  = []
  flow_p50PktSize = []
  flow_p90PktSize =  []
  rate = []
  flow_ipg_var = []
  flow_packet_size = []
  flow_packets = []
  flow_type = []
  flow_pkts_per_burst = []
  flow_ipg_mean = []
  flow_traffic = []
  link_capacity = []
  flow_length = []
  max_link_load = []
  path_to_link = []
  link_to_path = []
  delays =  []
  flow_bitrate_per_burst = []


  print(z_start, z)
  i = 0
  for train_lis in train_list[z_start:z]:



      ibgs.append(train_lis[0]["ibg"])
      flow_p10PktSize.append(np.array(train_lis[0]["flow_p10PktSize"]))
      flow_p50PktSize.append(np.array(train_lis[0]["flow_p50PktSize"]))
      flow_p90PktSize.append(np.array(train_lis[0]["flow_p90PktSize"]))
      rate.append(np.array(train_lis[0]["rate"]))
      flow_ipg_var.append(np.array(train_lis[0]["flow_ipg_var"]))
      flow_packet_size.append(np.array(train_lis[0]["flow_packet_size"]))
      flow_packets.append(np.array(train_lis[0]["flow_packets"]))
      flow_type.append(np.array(train_lis[0]["flow_type"]))
      flow_pkts_per_burst.append(np.array(train_lis[0]["flow_pkts_per_burst"]))
      flow_ipg_mean.append(np.array(train_lis[0]["flow_ipg_mean"]))
      flow_traffic.append(np.array(train_lis[0]["flow_traffic"]))
      flow_bitrate_per_burst.append(np.array(train_lis[0]["flow_bitrate_per_burst"]))

      link_capacity.append(np.array(train_lis[0]["link_capacity"]))
      flow_length.append(np.array(train_lis[0]["flow_length"]))

      # max_link_load.append(np.full(train_lis[0]["flow_traffic"].shape, np.array(train_lis[0]["max_link_load"]), ))
      max_link_load.append(np.array(train_lis[0]["max_link_load"]) )

      # path_to_link.append(np.array(train_lis[0]["path_to_link"]))


      path_to_link_ = train_lis[0]["path_to_link"].numpy()

      if i >0:
        # train_lis[0]["path_to_link"] = np.array(train_lis[0]["path_to_link"])[:]

        for path_to_lin in path_to_link_:
            path_to_lin[:,0] = path_to_lin[:,0]+ np.array(flow_traffic[-2]).shape[0]


        path_to_link.extend(path_to_link_)

        link_to_path.extend(  list(np.array(train_lis[0]["link_to_path"]) +  link_capacity[-2].shape[0]))
        # print(i, "second iteration", np.concatenate(link_capacity[-1]).shape[0], np.max(np.concatenate(link_to_path, axis =0)))
      else:


        link_to_path.extend( list(np.array(train_lis[0]["link_to_path"].numpy())))
        path_to_link.extend(list(path_to_link_))
        # print(i, "first iteration",  np.concatenate(link_capacity[-1]).shape[0], np.max(np.concatenate(link_to_path, axis =0)))
      delays.append(np.array(train_lis[1]))
      i+=1
      # print(train_lis[0]["devices"].shape
      # devices.append(train_lis[0]["devices"])

  z_start = z

  flow_traffic = np.concatenate(flow_traffic)
  flow_p10PktSize = np.concatenate(flow_p10PktSize)
  flow_p50PktSize = np.concatenate(flow_p50PktSize)
  flow_p90PktSize = np.concatenate(flow_p90PktSize)
  flow_pkts_per_burst = np.concatenate(flow_pkts_per_burst)
  flow_ipg_mean = np.concatenate(flow_ipg_mean)
  flow_ipg_var = np.concatenate(flow_ipg_var)
  ibgs = np.concatenate(ibgs)
  flow_length = np.concatenate(flow_length)
  link_capacity = np.concatenate(link_capacity)
  rate= np.concatenate(rate)
  flow_packet_size = np.concatenate(flow_packet_size)
  flow_packets = np.concatenate(flow_packets)
  flow_type = np.concatenate(flow_type)
  delays = np.concatenate(delays)
  flow_bitrate_per_burst = np.concatenate(flow_bitrate_per_burst)
  max_link_load = np.concatenate(max_link_load)
  loads = []
  normal_loads = []
 
 

  step_wise = len(path_to_link)/ max_link_load.shape[0]
  stepping = 0

  for m in range(len(path_to_link)):
      # print("m", torch.tensor(path_to_link[m][:, 0 ]).max(), len(flow_traffic))
      loads.append(torch.sum(torch.index_select((torch.tensor(flow_traffic)), 0, torch.tensor(path_to_link[m][:, 0 ]).type(torch.int64))) / (link_capacity[m] * 1e9))
      if m>0 and m % step_wise == 0:
        #  print("changing now", m  )
         stepping += 1
      normal_loads.append(loads[-1]/max_link_load[stepping])

  loads = torch.tensor(loads)
  normal_load = torch.tensor(normal_loads)
 

  # print(loads.shape,normal_load.shape )
  loads_capacities = torch.cat(((loads).unsqueeze(1), normal_load.unsqueeze(1), torch.tensor(link_capacity)), dim=1)


  width = flow_length.max()
  b = []

  for i in range(len(link_to_path)):
      if len(link_to_path[i]) != width:
          x = np.pad(link_to_path[i]+1, (0, width-len(link_to_path[i])), 'constant', constant_values=0)
      else:
          x = link_to_path[i]+1
      b.append(x)
  b = np.array(b)
  tensor_list = [torch.from_numpy(seq) for seq in path_to_link]
  tensor_lens = [len(seq) for seq in path_to_link]

  new_data.append(((flow_traffic, flow_p10PktSize,flow_p90PktSize, flow_pkts_per_burst, flow_ipg_mean, flow_ipg_var, ibgs, flow_length, link_capacity,rate, flow_packet_size,flow_packets,flow_type, link_to_path, path_to_link, max_link_load, flow_bitrate_per_burst, loads, tensor_lens, tensor_list, b, loads_capacities), delays))


0 2
2 4
4 6
6 8
8 10
10 12
12 14
14 16
16 18
18 20
20 22
22 24
24 26
26 28
28 30
30 32
32 34
34 36
36 38
38 40
40 42
42 44
44 46
46 48
48 50
50 52
52 54
54 56
56 58
58 60
60 62
62 64
64 66
66 68
68 70
70 72
72 74
74 76
76 78
78 80
80 82
82 84
84 86
86 88
88 90
90 92
92 94
94 96
96 98
98 100
100 102
102 104
104 106
106 108
108 110
110 112
112 114
114 116
116 118
118 120
120 122
122 124
124 126
126 128
128 130
130 132
132 134
134 136
136 138
138 140
140 142
142 144
144 146
146 148
148 150
150 152
152 154
154 156
156 158
158 160
160 162
162 164
164 166
166 168
168 170
170 172
172 174
174 176
176 178
178 180
180 182
182 184
184 186
186 188
188 190
190 192
192 194
194 196
196 198
198 200
200 202
202 204
204 206
206 208
208 210
210 212
212 214
214 216
216 218
218 220
220 222
222 224
224 226
226 228
228 230
230 232
232 234
234 236
236 238
238 240
240 242
242 244
244 246
246 248
248 250
250 252
252 254
254 256
256 258
258 260
260 262
262 264
264 266
266 268
268 270
270 272
272 274
274 276
276 

In [71]:
ibgs = []
flow_p10PktSize  = []
flow_p50PktSize = []
flow_p90PktSize =  []
rate = []
flow_ipg_var = []
flow_packet_size = []
flow_packets = []
flow_type = []
flow_pkts_per_burst = []
flow_bitrate_per_burst = []

flow_ipg_mean = []
flow_traffic = []
link_capacity = []
flow_length = []
max_link_load = []
path_to_link = []
link_to_path = []

delays =  []
batch_size = 512
# devices = []

new_data_val =[]
z_start = 0
for z in range(16, len(val_list[:]),16):
  ibgs = []
  flow_p10PktSize  = []
  flow_p50PktSize = []
  flow_p90PktSize =  []
  rate = []
  flow_ipg_var = []
  flow_packet_size = []
  flow_packets = []
  flow_type = []
  flow_pkts_per_burst = []
  flow_ipg_mean = []
  flow_traffic = []
  link_capacity = []
  flow_length = []
  max_link_load = []
  path_to_link = []
  link_to_path = []
  delays =  []
  flow_bitrate_per_burst = []


  print(z_start, z)
  i = 0
  for train_lis in train_list[z_start:z]:



      ibgs.append(train_lis[0]["ibg"])
      flow_p10PktSize.append(np.array(train_lis[0]["flow_p10PktSize"]))
      flow_p50PktSize.append(np.array(train_lis[0]["flow_p50PktSize"]))
      flow_p90PktSize.append(np.array(train_lis[0]["flow_p90PktSize"]))
      rate.append(np.array(train_lis[0]["rate"]))
      flow_ipg_var.append(np.array(train_lis[0]["flow_ipg_var"]))
      flow_packet_size.append(np.array(train_lis[0]["flow_packet_size"]))
      flow_packets.append(np.array(train_lis[0]["flow_packets"]))
      flow_type.append(np.array(train_lis[0]["flow_type"]))
      flow_pkts_per_burst.append(np.array(train_lis[0]["flow_pkts_per_burst"]))
      flow_ipg_mean.append(np.array(train_lis[0]["flow_ipg_mean"]))
      flow_traffic.append(np.array(train_lis[0]["flow_traffic"]))
      flow_bitrate_per_burst.append(np.array(train_lis[0]["flow_bitrate_per_burst"]))

      link_capacity.append(np.array(train_lis[0]["link_capacity"]))
      flow_length.append(np.array(train_lis[0]["flow_length"]))

      # max_link_load.append(np.full(train_lis[0]["flow_traffic"].shape, np.array(train_lis[0]["max_link_load"]), ))
      max_link_load.append(np.array(train_lis[0]["max_link_load"]) )

      # path_to_link.append(np.array(train_lis[0]["path_to_link"]))


      path_to_link_ = train_lis[0]["path_to_link"].numpy()

      if i >0:
        # train_lis[0]["path_to_link"] = np.array(train_lis[0]["path_to_link"])[:]

        for path_to_lin in path_to_link_:
            path_to_lin[:,0] = path_to_lin[:,0]+ np.array(flow_traffic[-2]).shape[0]


        path_to_link.extend(path_to_link_)

        link_to_path.extend(  list(np.array(train_lis[0]["link_to_path"]) +  link_capacity[-2].shape[0]))
        # print(i, "second iteration", np.concatenate(link_capacity[-1]).shape[0], np.max(np.concatenate(link_to_path, axis =0)))
      else:


        link_to_path.extend( list(np.array(train_lis[0]["link_to_path"].numpy())))
        path_to_link.extend(list(path_to_link_))
        # print(i, "first iteration",  np.concatenate(link_capacity[-1]).shape[0], np.max(np.concatenate(link_to_path, axis =0)))
      delays.append(np.array(train_lis[1]))
      i+=1
      # print(train_lis[0]["devices"].shape
      # devices.append(train_lis[0]["devices"])

  z_start = z

  flow_traffic = np.concatenate(flow_traffic)
  flow_p10PktSize = np.concatenate(flow_p10PktSize)
  flow_p50PktSize = np.concatenate(flow_p50PktSize)
  flow_p90PktSize = np.concatenate(flow_p90PktSize)
  flow_pkts_per_burst = np.concatenate(flow_pkts_per_burst)
  flow_ipg_mean = np.concatenate(flow_ipg_mean)
  flow_ipg_var = np.concatenate(flow_ipg_var)
  ibgs = np.concatenate(ibgs)
  flow_length = np.concatenate(flow_length)
  link_capacity = np.concatenate(link_capacity)
  rate= np.concatenate(rate)
  flow_packet_size = np.concatenate(flow_packet_size)
  flow_packets = np.concatenate(flow_packets)
  flow_type = np.concatenate(flow_type)
  delays = np.concatenate(delays)
  flow_bitrate_per_burst = np.concatenate(flow_bitrate_per_burst)
  max_link_load = np.concatenate(max_link_load)
  loads = []
  normal_loads = []
 
 

  step_wise = len(path_to_link)/ max_link_load.shape[0]
  stepping = 0

  for m in range(len(path_to_link)):
      # print("m", torch.tensor(path_to_link[m][:, 0 ]).max(), len(flow_traffic))
      loads.append(torch.sum(torch.index_select((torch.tensor(flow_traffic)), 0, torch.tensor(path_to_link[m][:, 0 ]).type(torch.int64))) / (link_capacity[m] * 1e9))
      if m>0 and m % step_wise == 0:
        #  print("changing now", m  )
         stepping += 1
      normal_loads.append(loads[-1]/max_link_load[stepping])

  loads = torch.tensor(loads)
  normal_load = torch.tensor(normal_loads)
 

  # print(loads.shape,normal_load.shape )
  loads_capacities = torch.cat(((loads).unsqueeze(1), normal_load.unsqueeze(1), torch.tensor(link_capacity)), dim=1)


  width = flow_length.max()
  b = []

  for i in range(len(link_to_path)):
      if len(link_to_path[i]) != width:
          x = np.pad(link_to_path[i]+1, (0, width-len(link_to_path[i])), 'constant', constant_values=0)
      else:
          x = link_to_path[i]+1
      b.append(x)
  b = np.array(b)
  tensor_list = [torch.from_numpy(seq) for seq in path_to_link]
  tensor_lens = [len(seq) for seq in path_to_link]

  new_data_val.append(((flow_traffic, flow_p10PktSize,flow_p90PktSize, flow_pkts_per_burst, flow_ipg_mean, flow_ipg_var, ibgs, flow_length, link_capacity,rate, flow_packet_size,flow_packets,flow_type, link_to_path, path_to_link, max_link_load, flow_bitrate_per_burst, loads, tensor_lens, tensor_list, b, loads_capacities), delays))


0 2
2 4
4 6
6 8
8 10
10 12
12 14
14 16
16 18
18 20
20 22
22 24
24 26
26 28
28 30
30 32
32 34
34 36
36 38
38 40
40 42
42 44
44 46
46 48
48 50
50 52
52 54
54 56
56 58
58 60
60 62
62 64
64 66
66 68
68 70
70 72
72 74
74 76
76 78
78 80
80 82
82 84
84 86
86 88
88 90
90 92
92 94
94 96
96 98
98 100
100 102
102 104
104 106
106 108
108 110
110 112
112 114
114 116
116 118
118 120
120 122
122 124
124 126
126 128
128 130
130 132
132 134
134 136
136 138
138 140
140 142
142 144
144 146
146 148
148 150
150 152
152 154
154 156
156 158
158 160
160 162
162 164
164 166
166 168
168 170
170 172
172 174
174 176
176 178
178 180
180 182
182 184
184 186
186 188
188 190
190 192
192 194
194 196
196 198
198 200
200 202
202 204
204 206
206 208
208 210
210 212
212 214
214 216
216 218
218 220
220 222
222 224
224 226
226 228
228 230
230 232
232 234
234 236
236 238
238 240
240 242
242 244
244 246
246 248
248 250
250 252
252 254
254 256
256 258
258 260
260 262
262 264
264 266
266 268
268 270
270 272
272 274
274 276
276 

In [72]:
import random
# random.shuffle(train_list)
train_dataset = CustomDataset(new_data)
val_dataset = CustomDataset(new_data_val)


In [73]:
normalizers = get_mean_std_dict(ds_train, mean_std_scores_fields)

In [74]:
import torch

class MeanAbsolutePercentageErrorLoss(torch.nn.Module):
    def __init__(self):
        super(MeanAbsolutePercentageErrorLoss, self).__init__()

    def forward(self, y_pred, y_true):
        """
        Compute the mean absolute percentage error between y_true and y_pred.

        Parameters:
            y_true (torch.Tensor): Ground truth values.
            y_pred (torch.Tensor): Predicted values.

        Returns:
            torch.Tensor: Mean absolute percentage error.
        """
        epsilon = 1e-7
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100


Transformer

In [75]:
import torch
import torch.nn.functional as F

import time
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
class PathEmbedding(nn.Module):
    def __init__(self, path_state_dim):
        super(PathEmbedding, self).__init__()
        self.path_state_dim = path_state_dim
        self.flow_embedding = nn.Sequential(
            nn.Linear(13, self.path_state_dim),
            nn.SELU(),
            nn.Linear(self.path_state_dim, self.path_state_dim),
            nn.SELU()
        )
        # self.initialize_weights()

    def initialize_weights(self):
        for layer in self.flow_embedding:
            if isinstance(layer, nn.Linear):
                # Use LeCun (Xavier) uniform initialization
                init.xavier_uniform_(layer.weight, gain=nn.init.calculate_gain('selu'))
                # init.constant_(layer.weight,1.0)

                if layer.bias is not None:
                    init.constant_(layer.bias, 0.0)
    def forward(self, x):
        return self.flow_embedding(x)

class LinkEmbedding(nn.Module):
    def __init__(self, link_state_dim):
        super(LinkEmbedding, self).__init__()
        self.link_state_dim = link_state_dim
        self.link_embedding = nn.Sequential(
            nn.Linear(3, self.link_state_dim),
            nn.SELU(),
            nn.Linear(self.link_state_dim, self.link_state_dim),
            nn.SELU()
        )
        self.initialize_weights()

    def initialize_weights(self):
        for layer in self.link_embedding:
            if isinstance(layer, nn.Linear):
                # Use LeCun (Xavier) uniform initialization
                init.xavier_uniform_(layer.weight, gain=nn.init.calculate_gain('selu'))
                # init.constant_(layer.weight,1.0)

                if layer.bias is not None:
                    init.constant_(layer.bias, 0.0)

    def forward(self, x):
        return self.link_embedding(x)

class PathReadout(nn.Module):
    def __init__(self, path_state_dim, link_state_dim):
        super(PathReadout, self).__init__()
        self.path_state_dim = path_state_dim
        self.link_state_dim = link_state_dim
        self.readout_path = nn.Sequential(
            nn.Linear(self.path_state_dim, self.link_state_dim // 2),
            nn.SELU(),
            nn.Linear(self.link_state_dim // 2, self.link_state_dim // 4),
            nn.SELU(),
            nn.Linear(self.link_state_dim // 4, 1),
            nn.Softplus()
        )
        # self.initialize_weights()

    def initialize_weights(self):
        for layer in self.readout_path:
            if isinstance(layer, nn.Linear):
                # Use LeCun (Xavier) uniform initialization
                init.xavier_uniform_(layer.weight, gain=nn.init.calculate_gain('selu'))
                # init.constant_(layer.weight,1.0)

                if layer.bias is not None:
                    init.constant_(layer.bias, 0.0)
    def forward(self, x):
        return self.readout_path(x)


class AttnModel(nn.Module):
    def __init__(self, path_state_dim):
        super(AttnModel, self).__init__()
        self.attn = nn.Sequential(
            nn.Linear(path_state_dim, path_state_dim),
            nn.LeakyReLU(negative_slope=0.01)
        )
        # Initialize the weights with zeros
        # self._initialize_weights()

    def _initialize_weights(self):
        for layer in self.attn:
            if isinstance(layer, nn.Linear):
                nn.init.ones_(layer.weight)
                #  init.xavier_uniform_(layer.weight, gain=nn.init.calculate_gain('selu'))
                if layer.bias is not None:
                    nn.init.ones_(layer.bias)


class CustomGRUModel(nn.Module):
    def __init__(self, path_state_dim):
        super(CustomGRUModel, self).__init__()
        self.gru = nn.GRU(path_state_dim, path_state_dim, batch_first=True)
        # Initialize the weights with zeros
        # self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.gru.named_parameters():
            if 'weight' in name:
                nn.init.ones_(param)
            elif 'bias' in name:
                nn.init.ones_(param)

In [80]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyCombinedModel(nn.Module):
    def __init__(self, link_state_dim, path_state_dim):
        super(MyCombinedModel, self).__init__()
        self.link_embed = LinkEmbedding(link_state_dim)  # Assuming LinkEmbedding is defined
        self.flow_embedded = PathEmbedding(path_state_dim)  # Assuming PathEmbedding is defined
        self.reader = PathReadout(path_state_dim, link_state_dim)  # Assuming PathReadout is defined
        self.link_update = nn.GRUCell(link_state_dim, link_state_dim)
        self.attention = AttnModel(path_state_dim).attn
        self.path_update = CustomGRUModel(path_state_dim).gru

    def forward(self, flow_length, flow_traffic_unnormalized, link_to_path, path_to_link, link_capacity, all_flows_used, flow_traffic, link_capacity_orig, max_link_load,  loads, tensor_lens, tensor_list, b, loads_capacities):
        all_flows_used = all_flows_used.to(device)
        path_state = self.flow_embedded(all_flows_used)
        flow_traffic_unnormalized = torch.tensor( flow_traffic_unnormalized).to(device)
        link_capacity = torch.tensor(link_capacity).to(device)
        flow_traffic = torch.tensor(flow_traffic).to(device)
        link_capacity_orig = torch.tensor(link_capacity_orig).to(device)


        loads_capacities = torch.tensor(loads_capacities).to(device)

        link_state = self.link_embed(loads_capacities)
        path_state = torch.unsqueeze(path_state, axis=0)



        for _ in range(12):
            link_state = torch.cat((torch.zeros(1, 16).to(device), link_state), dim=0)
            try:
                tensor_53x16 = link_state[b]
            except:
                continue

            path_state_sequence, path_state = self.path_update(tensor_53x16, path_state)
            idx = (torch.tensor(flow_length) - 1).view(-1, 1).expand(path_state_sequence.size(0), path_state_sequence.size(2)).unsqueeze(1).to(device)
            path_state = path_state_sequence.gather(1, idx.to(torch.int64)).squeeze(dim=1).unsqueeze(dim=0)
            prev_path_state = path_state.transpose(0, 1)
            path_state_sequence = torch.cat((prev_path_state, path_state_sequence), dim=1)
            path_state_sequence_for_indexing = torch.cat((torch.zeros_like(prev_path_state), path_state_sequence), dim=1)

            padded_path_to_link = nn.utils.rnn.pad_sequence(tensor_list, batch_first=True).to(device)
            path_sum = torch.sum(path_state_sequence_for_indexing[padded_path_to_link[:, :, 0], padded_path_to_link[:, :, 1]], axis=1)
            link_state = self.link_update(path_sum, link_state[1:, :])

        link_capacity_orig = torch.cat((torch.zeros(1, 1).to(device), (link_capacity_orig)), dim=0)
        capacity_gather_53x16 = link_capacity_orig[b]
        delays = []
        occupancy = self.reader(path_state_sequence[:, 1:])

        for i, path_state_seq in enumerate(path_state_sequence):
            delays.append(torch.sum(occupancy[i][:flow_length[i]][0] /(capacity_gather_53x16[i][:flow_length[i]]).T[0]).unsqueeze(0))

        return torch.cat(delays).to(device)


link_state_dim = 16
path_state_dim = 16

from torch.optim.lr_scheduler import ReduceLROnPlateau

model = MyCombinedModel(link_state_dim,  path_state_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5,patience=5)
criterion = MeanAbsolutePercentageErrorLoss()
mseloss_c = nn.MSELoss()
def train():
  model.train()
  # fake_batch_size= 1
  with tqdm(train_dataset, total=len(train_dataset)) as pbar:
    predictions = []
    y_all = []
    # loss = 0
    # loss = torch.tensor([0.0])

    for i,batch in enumerate(pbar):

        data, labels = batch  #

        ##flow_traffic 0, flow_p10PktSize 1,flow_p90PktSize 2, flow_pkts_per_burst 3, flow_ipg_mean 4, flow_ipg_var 5, ibgs 6,
        ##flow_length 7, link_capacity 8, rate 9, flow_packet_size 10,
        ##flow_packets 11 ,flow_type 12,link_to_path 13, path_to_link 14,
        ##max_link_load 15, flow_bitrate_per_burst 16,  loads  17, tensor_lens 18, tensor_list 19, b 20, load_capacities 21
        flow_traffic_unnormalized = data[0]
        flow_traffic = (data[0] - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
        flow_packets = (data[11]- normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
        max_link_load = data[15]
        flow_pkt_per_burst = (data[3] - normalizers["flow_pkts_per_burst"][0]) *normalizers["flow_pkts_per_burst"][1]
        flow_bitrate = (data[16] - normalizers["flow_bitrate_per_burst"][0]) *normalizers["flow_bitrate_per_burst"][1]
        flow_packet_size = (data[10]- normalizers["flow_packet_size"][0])  * normalizers["flow_packet_size"][1]
        flow_type = data[12]
        flow_ipg_mean = (data[4]- normalizers["flow_ipg_mean"][0])  * normalizers["flow_ipg_mean"][1]
        flow_ipg_var = (data[5] - normalizers["flow_ipg_var"][0])  * normalizers["flow_ipg_var"][1]

        cbr_rate = (data[9] - normalizers["rate"][0]) *normalizers["rate"][1]

        flow_length = data[7]
        ibg = (data[6] - normalizers["ibg"][0]) *normalizers["ibg"][1]
        flow_p90pktsize = (data[2] - normalizers["flow_p90PktSize"][0]) *normalizers["flow_p90PktSize"][1]

        link_capacity = (data[8] - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
        link_capacity_orig = data[8]
        link_to_path = data[13]
      
        path_to_link = data[14]
        flow_pkt_size_normal = flow_packets

        loads = data[17]
        tensor_lens = data[18]
        tensor_list = data[19]
        b = data[20]
        loads_capacities = data[21]
        # print(loads_capacities.shape)
        y_vals = torch.tensor(labels).to(device)



        all_flows_used = torch.tensor(np.concatenate([flow_traffic, ibg, cbr_rate, flow_p90pktsize, flow_packets, flow_type,  flow_packet_size, flow_bitrate, flow_ipg_mean, flow_ipg_var, flow_pkt_per_burst,    np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

        # all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )



        outputs = model(flow_length, flow_traffic_unnormalized, link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig,max_link_load, loads, tensor_lens, tensor_list, b, loads_capacities)

        predictions.extend(outputs.cpu().detach().numpy())
        # print("predictions",predictions[:5])

        loss = criterion(outputs, y_vals ) # Compute the loss.


        # if i%fake_batch_size == 0 or i== len(train_dataset):
        mseloss = torch.log(mseloss_c(outputs, y_vals))
        loss.backward()  # Derive gradients.
        # mseloss.backward()
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        # loss = torch.tensor([0.0])
        # print("outputs", outputs )
        pbar.set_description(f"Loss: {loss:.4f}  MSE {mseloss:.4f} lr: {scheduler.optimizer.param_groups[0]['lr']}")
        y_all.extend(labels)
    train_loss = criterion(torch.tensor(predictions), torch.tensor(y_all )).item()
    print("train loss = ",train_loss)
    # Print the gradients of the parameters before and after optimization step
    # for name, param in model.named_parameters():
    #     print(f'Parameter: {name}, Gradient Norm: {param.norm()}')

    model.eval()
    with tqdm(val_dataset, total=len(val_dataset)) as pbar:
      predictions = []
      y_all = []
      for batch in  pbar:
              # print(batch)
              # Process the batch as needed
          data, labels = batch  #

          ##flow_traffic 0, flow_p10PktSize 1,flow_p90PktSize 2, flow_pkts_per_burst 3, flow_ipg_mean 4, flow_ipg_var 5, ibgs 6,
          ##flow_length 7, link_capacity 8, rate 9, flow_packet_size 10,
          ##flow_packets 11 ,flow_type 12,link_to_path 13, path_to_link 14,
          ##max_link_load 15, flow_bitrate_per_burst 16,  loads  17, tensor_lens 18, tensor_list 19, b 20, load_capacities 21
        
          flow_traffic_unnormalized = data[0]

          flow_traffic = (data[0] - normalizers["flow_traffic"][0]) *normalizers["flow_traffic"][1]
          flow_packets = (data[11]- normalizers["flow_packets"][0]) *normalizers["flow_packets"][1]
          max_link_load = data[15]
          flow_pkt_per_burst = (data[3] - normalizers["flow_pkts_per_burst"][0]) *normalizers["flow_pkts_per_burst"][1]
          flow_bitrate = (data[16] - normalizers["flow_bitrate_per_burst"][0]) *normalizers["flow_bitrate_per_burst"][1]
          flow_packet_size = (data[10]- normalizers["flow_packet_size"][0])  * normalizers["flow_packet_size"][1]
          flow_type = data[12]
          flow_ipg_mean = (data[4]- normalizers["flow_ipg_mean"][0])  * normalizers["flow_ipg_mean"][1]
          flow_ipg_var = (data[5] - normalizers["flow_ipg_var"][0])  * normalizers["flow_ipg_var"][1]

          cbr_rate = (data[9] - normalizers["rate"][0]) *normalizers["rate"][1]

          flow_length = data[7]
          ibg = (data[6] - normalizers["ibg"][0]) *normalizers["ibg"][1]
          flow_p90pktsize = (data[2] - normalizers["flow_p90PktSize"][0]) *normalizers["flow_p90PktSize"][1]

          link_capacity = (data[8] - normalizers["link_capacity"][0]) *normalizers["link_capacity"][1]
          link_capacity_orig = data[8]
          link_to_path = data[13]
          path_to_link = data[14]
          flow_pkt_size_normal = flow_packets

          loads = data[17]
          tensor_lens = data[18]
          tensor_list = data[19]
          b = data[20]
          loads_capacities = data[21]

          y_vals = torch.tensor(labels).to(device)



          all_flows_used = torch.tensor(np.concatenate([flow_traffic, ibg, cbr_rate, flow_p90pktsize, flow_packets, flow_type,  flow_packet_size, flow_bitrate, flow_ipg_mean, flow_ipg_var, flow_pkt_per_burst,    np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )

          # all_flows_used = torch.tensor(np.concatenate([flow_traffic,flow_packets, flow_type, flow_type, np.expand_dims(flow_length,axis=1)], axis=1,dtype=np.float32 ) )



          outputs = model(flow_length, flow_traffic_unnormalized, link_to_path, path_to_link, link_capacity,all_flows_used, flow_traffic, link_capacity_orig,max_link_load, loads, tensor_lens, tensor_list, b, loads_capacities)

          val_loss_interm = criterion(outputs, y_vals)
          predictions.extend(outputs.cpu().detach().numpy())
          y_all.extend(labels)
          pbar.set_description(f"Loss: {val_loss_interm:.4f} lr: {scheduler.optimizer.param_groups[0]['lr']}")
    val_loss = criterion(torch.tensor(predictions), torch.tensor(y_all ))
    scheduler.step(val_loss.item())

    print("val loss = ",val_loss.item())

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, "checkpoint.pth")
    return train_loss, val_loss.item()
val_losses = []
tr_losses = []

for epoch in range(1, 150):
    t0 = time.time()
    print("epoch ", epoch)
    tr, vl = train()
    val_losses.append(vl)
    tr_losses.append(tr)

    print('{} seconds'.format(time.time() - t0))



epoch  1


  0%|          | 0/1689 [00:00<?, ?it/s]C:\Users\gudab\AppData\Local\Temp\ipykernel_15960\1752178708.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loads_capacities = torch.tensor(loads_capacities).to(device)
Loss: 98.1488  MSE 7.9789 lr: 0.001: 100%|██████████| 1689/1689 [1:11:22<00:00,  2.54s/it]  


train loss =  155.62538146972656


Loss: 40.4625 lr: 0.001: 100%|██████████| 422/422 [01:17<00:00,  5.43it/s]


val loss =  58.82002258300781
4360.18109536171 seconds
epoch  2


Loss: 80.1138  MSE 6.1613 lr: 0.001:  68%|██████▊   | 1151/1689 [17:13<07:23,  1.21it/s] 